<a href="https://colab.research.google.com/github/Anushaacharya607/Anushaacharya607.github.io/blob/master/maximum_count_keyword.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

General Strategy:
1.   Clean Data and tokenize
2.   Seperate korean and romans
3.   romans:Identify indonesia--> search in pickle, get lemma, get pos_tag
4.   else: pass through english spacy; get lemma; get pos_tag
5.   get korean pos_tag
6.   pass through individual stopwords
7.   get high freq words with pos_tag nn or jj












In [ ]:
# Hangul seperator
!pip install hangul-jamo

# Lemmatizer
!pip install -U spacy
!python -m spacy download en_core_web_sm
!python -m spacy download ko_core_news_sm
# load indo pickle
!pip install backports.lzma

!pip install wget


In [ ]:
import pandas as pd
# cleaning and tokenization
import re
import string

# Hangul seperator
import hangul_jamo

# Lemmatizer
import spacy
spe = spacy.load('en_core_web_sm')
spk = spacy.load('ko_core_news_sm')

#pickel
import pickle
try:
    import lzma
except ImportError:
    from backports import lzma
    
import wget

from collections import Counter




In [92]:
def pre_cleaning(text):
  tokens = []
  # Tokenization
  TOKREGEX = re.compile(
    r'(?:(?:[0-9][0-9.,:%-]*|St\.)[\w_€-]+|https?://[^ ]+|[@#§$]?\w[\w*_-]*|[,;:\.?!¿¡‽⸮…()\[\]–{}—―/‒_“„”⹂‚‘’‛′″‟\'"«»‹›<>=+−×÷•·]+)'
    )
  tokens = TOKREGEX.findall(text)
  tokens = list(filter(lambda token: token not in string.punctuation, tokens))
  # Double cleaning
  for token in tokens:
    for x in string.punctuation:
      if token.__contains__(x):
        tokens.remove(token)
        break
  # Lower
  tokens = list(map(lambda token: token.lower(), tokens))
  return tokens




In [4]:
def iswordKorean(word):
  characters = hangul_jamo.decompose(word)
  for character in characters:
    if not hangul_jamo.is_jamo_character(character):return False
  return True

In [22]:
def iswordID(word):
  stopwordsID = stopWordsID()
  id_dict =_load_pickle()
  if word in stopwordsID: return True
  elif id_dict.get(word): return True
  else: return False

In [8]:
def stopWordsID():
  filehandle = open("/content/StopwordsID.txt", "r")
  data = filehandle.read()
  stopwords = data.split('\n')
  filehandle.close()
  return stopwords

In [47]:
from spacy.util import working_dir
def lemmatizer(tokens):
  rows = []
  cols = ("text", "lemma", "POS","Stop Words")
  id_dict =_load_pickle()
  stopwords_ko = spk.Defaults.stop_words
  stopwords_en = spe.Defaults.stop_words
  for token in tokens:
    if iswordKorean(token):
      word = spk(token)
      for k in word:
        lemma = k.lemma_.split('+', 1)[0]
        pos = k.pos_
        if lemma in stopwords_ko:row = [k, lemma, pos, 1]
        else: row = [k, lemma, pos, 0]
    elif iswordID(token):
      ID_pos = ID_pos_()
      if token in ID_pos.values:
        pos = ID_pos.loc[ID_pos[0]==token].iloc[0][1]
      else: pos = 'NN'
      if token in stopWordsID():row=[token, token, pos, 1]
      elif id_dict.get(token): row=[token, id_dict.get(token), pos, 0]
  
    else:
      word = spe(token)
      for o in word:
        lemma = o.lemma_.split('+', 1)[0]
        pos = o.pos_
        if lemma.lower() in stopwords_en: row = [o, lemma, pos, 1]
        else:row = [o, lemma, pos, 0]
    rows.append(row)
  
  return pd.DataFrame(rows, columns=cols)



In [25]:
def _load_pickle():
  filepath = str("/content/id.plzma") 
  with lzma.open(filepath, "rb") as filehandle:
    pickled_dict = pickle.load(filehandle)
    assert isinstance(pickled_dict, dict)
    return pickled_dict

In [98]:
def ID_pos_():
  url ="https://raw.githubusercontent.com/famrashel/idn-tagged-corpus/master/Indonesian_Manually_Tagged_Corpus.tsv"
  Id_tag_corpus = wget.download(url)
  ID_pos=pd.read_csv(Id_tag_corpus,sep='\t', header=None)
  return ID_pos


In [159]:
def _Keyword(text, freq_limit=1, POS=['NOUN', 'NN'], num=None):
  filtered = pre_cleaning(text)
  df = lemmatizer(filtered)
  cd = df.loc[(df['Stop Words']== 0) & (df['POS'].isin(POS))]
  c = Counter({k: c for k, c in Counter(cd["lemma"]).items() if c > freq_limit})
  return c.most_common(num)  

In [160]:
text = ''' I am Anusha eating and again eating rice and I think I will go to fishing today and catch some fish. She loves it.
밥을 먹고 오늘은 낚시를 하러 갈 오늘 갈 것 같다.
Saya sedang makan nasi dan saya pikir saya akan pergi memancing hari ini.
I am eating and again eating rice and I think I will go to fishing today and catch some fish. She loves it.
밥을 먹고 오늘은 낚시를 하러 갈 오늘 갈 것 같다.
Saya sedang makan nasi dan saya pikir saya akan pergi memancing hari ini.'''
_Keyword(text, freq_limit=1, POS=['NOUN', 'NN', 'VB', 'VERB'] ) #num optional argument

[('eat', 4),
 ('오늘', 4),
 ('rice', 2),
 ('think', 2),
 ('fishing', 2),
 ('today', 2),
 ('catch', 2),
 ('fish', 2),
 ('love', 2),
 ('밥', 2),
 ('낚시', 2),
 ('makan', 2),
 ('nasi', 2),
 ('pikir', 2),
 ('pergi', 2),
 ('pancing', 2)]